In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt  
%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/arpitsinghrulania/kaggle/working/yolov5
Note: you may need to restart the kernel to use updated packages.
/home/arpitsinghrulania/kaggle/working
Setup complete. Using torch 1.13.0+cu117 (GeForce RTX 2080 Ti)


In [2]:
!pip3 install torch
import torch
from IPython.display import Image

In [3]:
# got some omp:error 15
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import time
import os
import sys
import shutil
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

import cv2

import torch
from torch.utils.data import Dataset, random_split
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import glob
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# from engine import train_one_epoch, evaluate
# import utils

from albumentations.pytorch.transforms import ToTensorV2

pd.options.display.max_colwidth = 100
print(torch.cuda.is_available())

BATCH_SIZE = 16
EPOCHS = 30
IMG_SIZE=1280
Selected_Fold=4



True


In [4]:
df = pd.read_csv('../train.csv')

In [5]:
# Get absolute path of root/parent directory
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(ROOT_DIR)
TRAIN_PATH = F'{ROOT_DIR}/train'
TEST_PATH = F'{ROOT_DIR}/test'
os.makedirs('COTS/images/train', exist_ok=True)
os.makedirs('COTS/images/valid', exist_ok=True)
os.makedirs('COTS/labels/train', exist_ok=True)
os.makedirs('COTS/labels/valid', exist_ok=True)

/home/arpitsinghrulania/kaggle


In [6]:
df['annotations'] = df['annotations'].apply(eval)
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'

In [7]:
df['num_bbox'] = df['annotations'].apply(lambda x: len(x))
bbox_proportion = (df.num_bbox>0).value_counts(normalize=True) * 100
print("Images with no bbox: {}%, Images with bbox: {}%".format(bbox_proportion[0], bbox_proportion[1]))

Images with no bbox: 79.06897578826432%, Images with bbox: 20.93102421173567%


In [8]:
df = df[df.num_bbox > 0].reset_index()

In [9]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

In [10]:
df['bboxes'] = df.annotations.apply(get_bbox)

## split into train and test

In [11]:
# train = df.sample(frac = 0.8, random_state=8)
# stratified sampling
train = df.groupby('num_bbox', group_keys=False).apply(lambda x: x.sample(frac=0.8))
test = df.drop(train.index)

In [12]:
train['image_path'] = f'{ROOT_DIR}/working/COTS/images/train/'+df.image_id+'.jpg'
train['label_path'] = f'{ROOT_DIR}/working/COTS/labels/train/'+df.image_id+'.txt'
test['image_path'] = f'{ROOT_DIR}/working/COTS/images/valid/'+df.image_id+'.jpg'
test['label_path'] = f'{ROOT_DIR}/working/COTS/labels/valid/'+df.image_id+'.txt'

## copy images to train and test folders

In [13]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

In [14]:
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _,row in tqdm(train.iterrows(), total=len(train)))

  0%|          | 0/3935 [00:00<?, ?it/s]

In [15]:
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy) (row) for _,row in tqdm(test.iterrows(), total=len(test)))

  0%|          | 0/984 [00:00<?, ?it/s]

## create annotation txt files

In [16]:
def coco2yolo(bboxes, height=720, width=1280):
    # normalising
    bboxes[..., 0::2] /= width
    bboxes[..., 1::2] /= height
    
    #conversion (xmin, ymin) -> (xmid, ymid)
    bboxes[..., 0:2] += bboxes[..., 2:4]/2
    return bboxes

def annot2str(data):
    data = data.astype(str)
    string = '\n'.join([' '.join(annot) for annot in data])
    return string

In [17]:
for i in tqdm(range(train.shape[0])):
    row = train.iloc[i]
    image_height = 720
    image_width = 1280
    bboxes_coco = np.array(row.bboxes).astype(np.float32).copy()
    labels = np.array([0]*row.num_bbox)[..., None].astype(str)
    ## Create Yolo Annotations txt file
    with open(row.label_path, 'w') as f:
        if row.num_bbox < 1:
            annot = ''
            f.write(annot)
            continue
        bboxes_yolo = coco2yolo(bboxes_coco, image_height, image_width)
        #print(bboxes_yolo)
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        #print(annots)
        string = annot2str(annots)
        #print(string)
        f.write(string)

  0%|          | 0/3935 [00:00<?, ?it/s]

In [18]:
for i in tqdm(range(test.shape[0])):
    row = test.iloc[i]
    image_height = 720
    image_width = 1280
    bboxes_coco = np.array(row.bboxes).astype(np.float32).copy()
    labels = np.array([0]*row.num_bbox)[..., None].astype(str)
    ## Create Yolo Annotations txt file
    with open(row.label_path, 'w') as f:
        if row.num_bbox < 1:
            annot = ''
            f.write(annot)
            continue
        bboxes_yolo = coco2yolo(bboxes_coco, image_height, image_width)
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        string = annot2str(annots)
        f.write(string)

  0%|          | 0/984 [00:00<?, ?it/s]

In [20]:
import yaml
with open(F'{ROOT_DIR}/working/train.txt', 'w') as f:
    for path in glob('/kaggle/working/train/*'):
        f.write(path+'\n')
            
with open(F'{ROOT_DIR}/working/val.txt', 'w') as f:
    for path in glob('/kaggle/working/val/*'):
        f.write(path+'\n')

data = dict(
    train = '../../COTS/images/train',
    val = '../../COTS/images/valid',
    
    nc    = 1, # number of classes
    names =  ['cots'] # classes
    )

with open(F'{ROOT_DIR}/working/yolov5/data/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

%cat /kaggle/working/yolov5/data/data.yaml

cat: /kaggle/working/yolov5/data/data.yaml: No such file or directory


In [21]:
%cd yolov5/

/home/arpitsinghrulania/kaggle/working/yolov5


In [23]:
!python3 train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data dataset.yaml \
                 --weights yolov5s.pt \
                 --project kaggle-Reef 

train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=kaggle-Reef, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-239-gf33718f Python-3.9.12 torch-1.13.0+cu117 CUDA:0 (GeForce RTX 2080 Ti, 11016MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.